In [11]:
import pandas as pd
import json

# Load the data
df = pd.read_csv(r'C:\Users\mcaballero\Desktop\pro22_0364_p3di_tecnara_aei2022b_mincotur_mrr\resources\dataset\data_imascono.csv')

# Transform user_id from string to numeric values
user_ids = df['user_id'].unique()
user_id_dict = {user_id: i+1 for i, user_id in enumerate(user_ids)}
df['user_id'] = df['user_id'].map(user_id_dict)

# user.csv
df_user = df[['user_id']].drop_duplicates()
df_user.to_csv(r'C:\Users\mcaballero\Desktop\pro22_0364_p3di_tecnara_aei2022b_mincotur_mrr\resources\dataset\user.csv', index=False)

# item.csv
df_item = df[df['object_type'] != 'Player'][['object_id', 'object_type', 'object_action', 'object_position', 'room_id']].drop_duplicates()
df_item.columns = ['item_id', 'object_type', 'object_action_types', 'object_position', 'room_id']

grouped_data = df_item.groupby('object_type')['object_action_types'].agg(lambda x: x.unique())
for object_type, object_actions in grouped_data.items():
    df_item.loc[df_item['object_type'] == object_type, 'object_action_types'] = ', '.join(object_actions)

df_item.to_csv(r'C:\Users\mcaballero\Desktop\pro22_0364_p3di_tecnara_aei2022b_mincotur_mrr\resources\dataset\item.csv', index=False)

# context.csv
df_context = df[['device_data']].drop_duplicates()
df_context['device_data'] = df_context['device_data'].apply(lambda x: str(json.loads(x)) if pd.notnull(x) else "{}")
df_context = df_context.reset_index()
df_context.columns = ['context_id', 'device_data']
df_context.to_csv(r'C:\Users\mcaballero\Desktop\pro22_0364_p3di_tecnara_aei2022b_mincotur_mrr\resources\dataset\context.csv', index=False)

# behavior.csv
df_behavior = df[['user_id', 'object_action', 'object_position', 'object_id', 'device_data', '@timestamp']].dropna()
df_behavior.columns = ['user_id', 'object_action', 'user_position', 'item_id', 'device_data', 'timestamp']

# Merge behavior.csv and context.csv on device_data to ensure context_id is consistent
df_behavior = pd.merge(df_behavior, df_context, how='left', on='device_data')
df_behavior.drop(columns=['device_data'], inplace=True)

df_behavior['behavior_id'] = df_behavior.index + 1
df_behavior.to_csv(r'C:\Users\mcaballero\Desktop\pro22_0364_p3di_tecnara_aei2022b_mincotur_mrr\resources\dataset\behavior.csv', index=False)